In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
def mask(column,threshold):
    counts = pd.value_counts(column)
    mask = column.isin(counts[counts > threshold].index)
    column[~mask] = "others"
    return column

In [3]:
bookings_data = pd.read_csv('BookingsData.csv')

In [4]:
bookings_data = bookings_data.drop_duplicates()
bookings_data["Customer_Name"] = bookings_data["Customer_Name"].str.lower()
bookings_data["Product_Family"] = bookings_data["Product_Family"].str.lower()
bookings_data["BOARD_GEO"] = bookings_data["BOARD_GEO"].str.lower()
bookings_data["Customer_Name"] = bookings_data["Customer_Name"].replace('\s+', ' ', regex=True)
bookings_data["TX_Date_mod"] = pd.to_datetime(bookings_data['TX_Date'])
bookings_data = bookings_data[(bookings_data["TX_Date_mod"] > '2012')]
bookings_data.loc[(bookings_data['Product_Family'] == "lan desk (other)"),"Product_Family"] = 'others'

# if trying classification
bookings_data['Product_Family'] = mask(bookings_data['Product_Family'], 50)
bookings_data['PRODUCTCODE'] = mask(bookings_data['PRODUCTCODE'], 20)
bookings_data.count()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Customer_Name      141799
Account_ID          14265
Bookings Amount    141799
TX_Date            141799
Seats              141799
GEO                 98553
BOARD_GEO          141799
VERTICAL           125251
Product_Family     141799
Deal Type          141799
Product Type       141799
Product Segment    135956
Contract Start     113938
Contract End        89477
PRODUCTCODE        141799
FAMILY             139802
NAME               141799
DESCRIPTION         87502
TX_Date_mod        141799
dtype: int64

In [5]:
bookings_data['Seats_mod'] = np.where(bookings_data['Seats'] > 0, 1, (np.where(bookings_data['Seats'] == 0, 0, -1)))

In [6]:
company_standards = pd.read_csv('standardization2.csv')
company_standards_2 = pd.read_csv('standardization2_2.csv')
print(company_standards.shape)

bookings_data = bookings_data.merge(company_standards, left_on='Customer_Name', right_on='Similar_Name', how='outer')
bookings_data = bookings_data.rename(index=str, columns={"Customer_Name_x": "Customer_Name_old1", 
                                                         "Customer_Name_y": "Customer_Name"})

bookings_data = bookings_data.merge(company_standards_2, left_on='Customer_Name', right_on='Similar_Name', how='outer')
bookings_data = bookings_data.rename(index=str, columns={"Customer_Name_x": "Customer_Name_old2", 
                                                         "Customer_Name_y": "Customer_Name"})

bookings_data["Customer_Name"] = np.where(bookings_data.Customer_Name.isna(), 
                    np.where(bookings_data.Customer_Name_old2.isna(), bookings_data.Customer_Name_old1 , bookings_data.Customer_Name_old2),
                                       bookings_data.Customer_Name)   

print(len(bookings_data.Customer_Name.unique()))
print(len(bookings_data.Customer_Name_old2.unique()))
print(len(bookings_data.Customer_Name_old1.unique()))
print(bookings_data.shape)

(2572, 3)
21762
1308
23664
(149892, 26)


In [ ]:
a = bookings_data.Customer_Name.unique()
for i in a:
    print(i)
#     print("\n--------\n")

In [7]:
table = bookings_data.pivot_table(values=['Seats_mod'], index=['Customer_Name'], columns=['Product_Family'],
                     aggfunc='sum').fillna(0)
table

Seats_mod  \
Product_Family                                     anti virus   
Customer_Name                                                   
 interceramic                                             0.0   
 movitex                                                  1.0   
 oao bank petrocommerz                                    0.0   
##(##)############                                        0.0   
#eská lékárna a.s.                                        0.0   
's heeren loo                                             0.0   
't sit                                                    0.0   
(bet365) hillside (new media) ltd                         0.0   
(dhhs) us department of health &                          0.0   
(dhhs) us department of health & human services           0.0   
(fcos)                                                    0.0   
**radiant systems-do not use                              1.0   
0                                                         0.0   
1-800 east west mortgage                                  0.0   
1-net singapore pte ltd                                   0.0   
102 israel fire and rescue service                        0.0   
1199nbf                                                   0.0   
1199seiu l benefit & pension f                            0.0   
121 financial credit union                                0.0   
1469-kona surf partners                                   0.0   
1st alliance lending                                      0.0   
1st alliance lending, llc                                 0.0   
1st global                                                0.0   
1st midamerica credit union                               0.0   
1st national bank of south florida                        0.0   
1st partners & co., inc.                                  0.0   
1st source bank                                           0.0   
1st united bank                                           0.0   
2 sisters food group                                      0.0   
20 minutes france sas                                     6.0   
...                                                       ...   
zunhammer gmbh                                            0.0   
zurcher hichschuke fur angewandte                         0.0   
zurcher hichschuke fur angewandte wissens                 0.0   
zurich academy of applied sciences wissens                0.0   
zurich compania de seguros s.a.                           0.0   
zurich financial services (iom) gro                       0.0   
zurich financial services (iom) group                     0.0   
zurich financial services australia limited               0.0   
zurich hochschule der kuenste                             0.0   
zurich insurance                                          0.0   
zuwe hofpoort ziekenhuis                                  0.0   
zuyderland mc                                             0.0   
zuyderland medisch centrum                                0.0   
zv kommunale verkehrssicherheit                           0.0   
zv kommunale verkehrssicherheit obe                       0.0   
zv kommunale verkehrssicherheit oberland                  0.0   
zvei zentralverband elektrotechnik-und elektron...        0.0   
zweckverband abfallverwertung südostbayern                0.0   
zweckverband kommunale verkehrsüberwachung südo...        0.0   
zwembad de meerval                                        0.0   
zwick gmbh & co kg                                        0.0   
zwicker & associates                                      7.0   
zwp ingenieur-ag                                          0.0   
zynga                                                     0.0   
zürcher kantonalbank (zkb)                                0.0   
öbb ikt gmbh                                              0.0   
östereichischer verband gemeinnütziger bauverei...        0.0   
österreich werbung                                        0.0   
österreichische präsid

In [8]:
a = bookings_data[['Customer_Name','BOARD_GEO']]
a = a.drop_duplicates()
a = a.reset_index()
a.head()

,index,Customer_Name,BOARD_GEO
0,0,interceramic,americas
1,1,movitex,emea
2,2,oao bank petrocommerz,emea
3,5,büroring personalmanagement gmbh,emea
4,7,##(##)############,china


In [9]:
a["Customer_Name"].describe()

count     22238
unique    21762
top         ing
freq          4
Name: Customer_Name, dtype: object

In [17]:
s_words = ['of','inc','ltd','limited','co','llc','group','the','corp','corporation','comp','company','gmbh','bank'
           ,'credit','union','county','school','university','hospital','district','mexico','de','sa','cv','bv'
           ,'services','federal','city','state']
no_features = 100
# tf_vectorizer = CountVectorizer(ngram_range = (1,1),analyzer='word'
#                                  , max_df=0.005
#  #                             ,tokenizer=lambda x: x.split(' ')
#  #                              ,stop_words=s_words
#                                )
tf_vectorizer = TfidfVectorizer(ngram_range = (1,2),analyzer='word'
                                ,token_pattern=r"(?u)\S\S+"
#                                  , max_df=0.005
 #                             ,tokenizer=lambda x: x.split(' ')
 #                              ,stop_words=s_words
                               )
tf = tf_vectorizer.fit_transform(a['Customer_Name'])

tf_feature_names = tf_vectorizer.get_feature_names()
print(len(tf_feature_names))
weights = np.asarray(tf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tf_feature_names, 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(50)

57871


,term,weight
37577,of,0.012253
22370,gmbh,0.009054
5924,bank,0.008827
26431,inc.,0.007294
32001,ltd,0.007223
46297,services,0.006257
26409,inc,0.005637
22979,group,0.005633
2291,ag,0.005295
13013,corporation,0.005106


In [11]:
a.loc[4]

index                             7
Customer_Name    ##(##)############
BOARD_GEO                     china
Name: 4, dtype: object

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
cos_similarity_matrix = (tf * tf.T).toarray()

In [19]:
print(cos_similarity_matrix[0:10])

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [21]:
ind = np.argwhere(cos_similarity_matrix > 0)
# print(len(ind))
count = 0
standard = []
for i,j in ind:
    if i < j:
        c = len(set(a.loc[i]['Customer_Name'].split()))
        d = cos_similarity_matrix[i][j] 
        sim = d/c
#         print(sim)
        if sim > 0.4: #always check the condition
            standard.append((a.loc[i]['Customer_Name'],a.loc[j]['Customer_Name']))
#             print(a.loc[i]['Customer_Name'], sim, a.loc[j]['Customer_Name'], c, d)
#             count += 1
#             if count > 100:
#                 break


In [22]:
standard_df = pd.DataFrame(standard, columns=['Customer_Name','Similar_Name'])

In [23]:
standard_df.to_csv('Standardization_test.csv')

In [32]:
from scipy.sparse import csr_matrix, find
b = find(cos_similarity_matrix)

In [ ]:
# from nltk.corpus import stopwords
# from gensim.models.doc2vec import TaggedDocument
# from gensim import utils
# import re
# import string
# from gensim.models import Doc2Vec

# def constructLabeledSentences(data):
#     sentences=[]
#     for index, row in data.iteritems():
#         sentences.append(TaggedDocument(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
#     return sentences

# def textClean(text):
#     text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
#     text = text.lower().split()
#     stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops]    
#     text = " ".join(text)
#     return(text)
    
# def cleanup(text):
#     text = textClean(text)
#     text= text.translate(str.maketrans("","", string.punctuation))
#     return text

# allText = bookings_data['Customer_Name'].apply(cleanup)
# sentences = constructLabeledSentences(allText)
# allText.head()

In [ ]:
# model = Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
# model.build_vocab(sentences)
# for epoch in range(10):
#     model.train(sentences,total_examples=model.corpus_count,epochs=model.epochs)
#     model.alpha -= 0.002  # decrease the learning rate
#     model.min_alpha = model.alpha  # fix the learning rate, no decay

In [ ]:
# print(model.most_similar(&quot;allText[0]&quot;))

In [ ]:
# from difflib import SequenceMatcher

# def similar(a, b):
#     return SequenceMatcher(None, a, b).ratio()

In [ ]:
# from difflib import get_close_matches
# get_close_matches('oao bank petrocommerz', a)

In [ ]:
# b = list(a)
# un = {}
# for i in b:
#     temp = get_close_matches(i, b, n=100, cutoff=0.8)
#     if len(temp) > 0:
#         temp.append(i)
#         un[i] = temp
#         b = [e for e in b if e not in temp]        

In [ ]:
bookings_data.head(5)

In [ ]:
bookings_data["TX_Date_mod"] = pd.to_datetime(bookings_data['TX_Date'])

In [ ]:
bookings_data.loc[(bookings_data['Product_Family'] == "lan desk (other)"),"Product_Family"] = 'others'

In [ ]:
bookings_data['Product_Family'].value_counts()

In [137]:
bookings_data[(bookings_data['Customer_Name'].str.contains("hewlett"))]

,Customer_Name,Account_ID,Bookings Amount,TX_Date,Seats,GEO,BOARD_GEO,VERTICAL,Product_Family,Deal Type,Product Type,Product Segment,Contract Start,Contract End,PRODUCTCODE,FAMILY,NAME,DESCRIPTION,TX_Date_mod,Seats_mod
103053,hewlett - packard (hp-psg),NaN,-263196.2200,2014-09-05 00:00:00.000,-1,SH-WW,americas,Standard,oem licenced royalties,Volume,OEM,Service NonProduct,2014-09-05 00:00:00.000,2015-09-04 00:00:00.000,OEM-LDROYALTY,Other Lic\Mnt\Sub\OEM,JSI OEM Royalty,NaN,2014-09-05,-1
103054,hewlett - packard (hp-psg),NaN,263196.2200,2014-09-05 00:00:00.000,1,SH-WW,americas,Standard,oem licenced royalties,Volume,OEM,Service NonProduct,2014-09-05 00:00:00.000,2015-09-04 00:00:00.000,OEM-LDROYALTY,Other Lic\Mnt\Sub\OEM,JSI OEM Royalty,NaN,2014-09-05,1
103055,hewlett - packard (hp-psg) hp contract -busine...,NaN,30299.6000,2015-03-31 00:00:00.000,2110,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
103056,hewlett - packard (hp-psg) hp contract -busine...,NaN,1048.2800,2015-03-31 00:00:00.000,73,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
103057,hewlett - packard (hp-psg) hp contract -busine...,NaN,2154.0000,2015-03-31 00:00:00.000,150,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
103058,hewlett - packard (hp-psg) hp contract -busine...,NaN,1723.2000,2015-03-31 00:00:00.000,120,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
103059,hewlett - packard (hp-psg) hp contract -busine...,NaN,258.4800,2015-03-31 00:00:00.000,18,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
103060,hewlett - packard (hp-psg) hp contract -busine...,NaN,1436.0000,2015-03-31 00:00:00.000,100,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
103061,hewlett - packard (hp-psg) hp contract -busine...,NaN,2872.0000,2015-03-31 00:00:00.000,200,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
103062,hewlett - packard (hp-psg) hp contract -busine...,NaN,789.8000,2015-03-31 00:00:00.000,55,LAR,americas,Standard,ldim (inventory manager),New,LIC,Software,2015-03-31 00:00:00.000,NaN,LDIM-L,Inventory Manager,"Ivanti Inventory Manager, powered by Landesk",NaN,2015-03-31,1
